# VacationPy
----

In [1]:
%%html
<style>
    .jupyter-widgets-disconnected:before { display: none; }
</style>

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Load the csv exported in Part I to a DataFrame
cities_df = pd.read_csv(os.path.join("output_data","cities.csv"))
cities_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Puerto Ayora,20,EC,1584796244,94,-0.74,-90.35,78.01,5.82
1,1,Necochea,0,AR,1584796538,69,-38.55,-58.74,73.00,10.00
2,2,Constitución,100,CL,1584796252,81,-35.33,-72.42,55.31,10.13
3,3,Ushuaia,100,AR,1584796206,87,-54.80,-68.30,42.80,4.70
4,4,Hilo,90,US,1584796245,83,19.73,-155.09,71.60,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(g_key)
map = gmaps.figure(zoom_level=2, center=(20,0))

# Use the Lat and Lng as locations and Humidity as the weight
locations = cities_df[["Lat", "Lng"]].to_records(index=False)
weight = cities_df["Humidity"].to_list()

# Add Heatmap layer to map
map.add_layer(gmaps.heatmap_layer(locations, weight, point_radius=20))
map

Figure(layout=FigureLayout(height='420px'))

In [5]:
cities_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Puerto Ayora,20,EC,1584796244,94,-0.74,-90.35,78.01,5.82
1,1,Necochea,0,AR,1584796538,69,-38.55,-58.74,73.00,10.00
2,2,Constitución,100,CL,1584796252,81,-35.33,-72.42,55.31,10.13
3,3,Ushuaia,100,AR,1584796206,87,-54.80,-68.30,42.80,4.70
4,4,Hilo,90,US,1584796245,83,19.73,-155.09,71.60,6.93


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
narrowed_city_df = cities_df[
    # A max temperature lower than 80 degrees
    (cities_df["Max Temp"] < 80) &
    #  but higher than 70.
    (cities_df["Max Temp"] > 70) &
    # Wind speed less than 10 mph.
    (cities_df["Wind Speed"] < 10) &
    # Zero cloudiness.
    (cities_df["Cloudiness"] == 0)
].copy()

narrowed_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
33,33,Imbituba,0,BR,1584796229,56,-28.24,-48.67,73.40,3.36
37,37,Port Alfred,0,ZA,1584796247,79,-33.59,26.89,73.00,1.99
41,41,Geraldton,0,AU,1584796322,83,-28.77,114.60,73.40,8.05
47,47,Alice Springs,0,AU,1584796212,29,-23.70,133.88,78.80,4.70
203,203,Mar del Plata,0,AR,1584796255,100,-38.00,-57.56,72.00,6.93
211,211,Adana Province,0,TR,1584796680,28,36.98,35.29,71.60,3.36
387,387,Treinta y Tres,0,UY,1584796751,51,-33.23,-54.38,70.25,5.44
391,391,Walvis Bay,0,NaN,1584796373,57,-22.96,14.51,75.20,8.05
416,416,Morón,0,AR,1584796743,64,-34.65,-58.62,75.20,5.82
559,559,Rocha,0,UY,1584796286,54,-34.48,-54.33,70.97,4.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Store into variable named hotel_df
hotel_df = pd.DataFrame({
    # Add a "Hotel Name" column to the DataFrame
    "Hotel Name": [],
    "Lat": [],
    "Lng": []
})

# Set parameters to search for hotels with 5000 meters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = { 
    "types": "lodging",
    "radius": "5000",
    "key": g_key,
}

# Hit the Google Places API for each city's coordinates
for index, row in narrowed_city_df.iterrows():
    params["location"] = f"{row['Lat']},{row['Lng']}"
    hotel = requests.get(base_url, params = params).json()
    try:
        # Store the first Hotel result into the DataFrame
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
        narrowed_city_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
        hotel_df.loc[index, "Lat"] = hotel["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Lng"] = hotel["results"][0]["geometry"]["location"]["lng"]
    except IndexError:
        pass # Welp, guess there's nowhere to stay in this lovely city

hotel_df.head()

,Hotel Name,Lat,Lng
33,Praia Hotel Imbituba,-28.246528,-48.666917
37,The Halyards Hotel,-33.594329,26.893248
41,Broadwater Mariner Resort,-28.749328,114.619364
47,Desert Palms Alice Springs,-23.713395,133.879820
203,Gran Hotel Mar del Plata habitaciones,-38.001965,-57.543790


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
map.add_layer(gmaps.marker_layer(locations, info_box_content = hotel_info))

# Display Map
map

Figure(layout=FigureLayout(height='420px'))